##### Copyright 2021 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# 使用 TFF 的 ClientData

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/federated/tutorials/working_with_client_data"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">在 TensorFlow.org 上查看</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/federated/blob/v0.34.0/docs/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">在 Google Colab 中运行</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/federated/blob/v0.34.0/docs/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">在 GitHub 上查看源代码</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/federated/docs/tutorials/working_with_client_data.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">下载笔记本</a>   </td>
</table>

由客户端（例如用户）键入数据集的概念对于在 TFF 中建模的联合计算至关重要。TFF 提供了接口 [`tff.simulation.datasets.ClientData`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData) 来抽象化这个概念，而 TFF 所托管的数据集 ([stackoverflow](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/stackoverflow)、[shakespeare](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/shakespeare)、[emnist](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/emnist)、[cifar100](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/cifar100) 和 [gldv2](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/gldv2)) 都实现了这个接口。

如果您正在使用自己的数据集进行联合学习，TFF 强烈建议您实现 `ClientData` 接口或使用其中一个 TFF 的帮助函数来生成表示磁盘上数据的 `ClientData`，例如 [`tff.simulation.datasets.ClientData.from_clients_and_fn`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData#from_clients_and_fn)。

由于 TFF 的大多数端到端示例都从 `ClientData` 对象开始，因此使用您的自定义数据集实现 `ClientData` 接口将使您更容易深入研究用 TFF 编写的现有代码。此外，可以直接迭代由 `ClientData` 构造的 `tf.data.Datasets`，以生成 `numpy` 数组的结构，因此在迁移到 TFF 之前，`ClientData` 对象可以与任何基于 Python 的 ML 框架一起使用。

如果您打算将模拟扩展到多台计算机或部署它们，有几种模式可以让您的工作变得更加容易。下面，我们将介绍几种使用 `ClientData` 和 TFF 的方式，以使我们的小规模迭代-到大规模实验-到生产部署体验尽可能顺利。

## 我应该使用哪种模式将 ClientData 传递到 TFF？

我们将深入讨论 TFF 的 `ClientData` 的两个用法；如果您符合以下两个情况中的任何一种，您显然会更喜欢其中某一种用法。如果不是，您可能需要更详细地了解每种用法的利弊，以便做出更细微的选择。

- 我希望在本地计算机上尽可能快速地迭代；我不需要能够轻松地利用 TFF 的分布式运行时。

    - 您希望将 `tf.data.Datasets` 直接传入 TFF。
    - 这允许您使用 `tf.data.Dataset` 对象进行强制编程，并任意处理它们。
    - 它提供了比下面的选项更大的灵活性；将逻辑推送到客户端要求该逻辑可序列化。

- 我想在 TFF 的远程运行时中运行我的联合计算，或者我计划会很快这样做。

    - 在这种情况下，您希望将数据集的构造和预处理映射到客户端。
    - 这会导致您直接将包含 `client_ids` 的列表传递给您的联合计算。

- 将数据集构造和预处理推送到客户端能够避免序列化中的瓶颈，并显著提高成百上千个客户端的性能。

In [ ]:
#@title Set up open-source environment
#@test {"skip": true}

# tensorflow_federated_nightly also bring in tf_nightly, which
# can causes a duplicate tensorboard install, leading to errors.
!pip uninstall --yes tensorboard tb-nightly

!pip install --quiet --upgrade tensorflow_federated
!pip install --quiet --upgrade nest_asyncio

import nest_asyncio
nest_asyncio.apply()

In [1]:
#@title Import packages
import collections
import time

import tensorflow as tf
import tensorflow_federated as tff

## 操作 ClientData 对象

我们从加载和探索 TFF EMNIST `ClientData` 开始：


In [2]:
client_data, _ = tff.simulation.datasets.emnist.load_data()

通过检查第一个数据集，我们可以知道  `ClientData` 中的示例是什么类型。

In [3]:
first_client_id = client_data.client_ids[0]
first_client_dataset = client_data.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)
# This information is also available as a `ClientData` property:
assert client_data.element_type_structure == first_client_dataset.element_spec

OrderedDict([('label', TensorSpec(shape=(), dtype=tf.int32, name=None)), ('pixels', TensorSpec(shape=(28, 28), dtype=tf.float32, name=None))])


请注意，数据集会生成具有 `pixels` 和 `label` 键的 `collections.OrderedDict` 对象，其中 pixels 是形状为 `[28, 28]` 的张量。假设我们希望将输入展平为形状 `[784]`。一种可能的方式是对 `ClientData` 对象应用预处理函数。

In [4]:
def preprocess_dataset(dataset):
  """Create batches of 5 examples, and limit to 3 batches."""

  def map_fn(input):
    return collections.OrderedDict(
        x=tf.reshape(input['pixels'], shape=(-1, 784)),
        y=tf.cast(tf.reshape(input['label'], shape=(-1, 1)), tf.int64),
    )

  return dataset.batch(5).map(
      map_fn, num_parallel_calls=tf.data.experimental.AUTOTUNE).take(5)


preprocessed_client_data = client_data.preprocess(preprocess_dataset)

# Notice that we have both reshaped and renamed the elements of the ordered dict.
first_client_dataset = preprocessed_client_data.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)

OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])


我们可能还需要执行一些更复杂的（可能是有状态的）预处理，例如混排。

In [5]:
def preprocess_and_shuffle(dataset):
  """Applies `preprocess_dataset` above and shuffles the result."""
  preprocessed = preprocess_dataset(dataset)
  return preprocessed.shuffle(buffer_size=5)

preprocessed_and_shuffled = client_data.preprocess(preprocess_and_shuffle)

# The type signature will remain the same, but the batches will be shuffled.
first_client_dataset = preprocessed_and_shuffled.create_tf_dataset_for_client(
    first_client_id)
print(first_client_dataset.element_spec)

OrderedDict([('x', TensorSpec(shape=(None, 784), dtype=tf.float32, name=None)), ('y', TensorSpec(shape=(None, 1), dtype=tf.int64, name=None))])


## 与 `tff.Computation` 对接

现在我们可以对 `ClientData` 对象执行一些基本操作，接下来可以将数据馈送给 `tff.Computation`。我们定义一个 [`tff.templates.IterativeProcess`](https://www.tensorflow.org/federated/api_docs/python/tff/templates/IterativeProcess)，它能够实现 [Federated Averaging](https://arxiv.org/abs/1602.05629)，并能够探索传递数据的不同方法。

In [6]:
def model_fn():
  model = tf.keras.models.Sequential([
      tf.keras.layers.InputLayer(input_shape=(784,)),
      tf.keras.layers.Dense(10, kernel_initializer='zeros'),
  ])
  return tff.learning.from_keras_model(
      model,
      # Note: input spec is the _batched_ shape, and includes the 
      # label tensor which will be passed to the loss function. This model is
      # therefore configured to accept data _after_ it has been preprocessed.
      input_spec=collections.OrderedDict(
          x=tf.TensorSpec(shape=[None, 784], dtype=tf.float32),
          y=tf.TensorSpec(shape=[None, 1], dtype=tf.int64)),
      loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
      metrics=[tf.keras.metrics.SparseCategoricalAccuracy()])
  
trainer = tff.learning.algorithms.build_weighted_fed_avg(
    model_fn,
    client_optimizer_fn=lambda: tf.keras.optimizers.SGD(learning_rate=0.01))

在我们开始使用 `IterativeProcess` 之前，有一条关于 `ClientData` 语义的注释。`ClientData` 对象表示可用于联合训练的群体的 *整体*，其通常[不可用于生产 FL 系统的执行环境](https://arxiv.org/abs/1902.01046)，并且特定于模拟。`ClientData` 确实为用户提供了完全绕过联合计算的能力，并可以像往常一样直接通过 [`ClientData.create_tf_dataset_from_all_clients`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/datasets/ClientData?hl=en#create_tf_dataset_from_all_clients) 训练服务器端模型。

TFF 的模拟环境使研究人员能够完全控制外部循环。尤其是，这意味着客户端可用性、客户端退出等方面的考虑必须由用户或 Python 驱动程序脚本来解决。例如，可以通过调整 `ClientData` 的 `client_ids` 上的采样分布来模拟客户端退出，从而以较低的概率选择具有更多数据（且相应地具有更长的本地计算运行时间）的用户。

然而，在实际的联合系统中，客户端无法由模型 trainer 显式选择；客户端的选择会委托给执行联合计算的系统。

### 将 `tf.data.Datasets` 直接传递给 TFF

在 `ClientData` 和 `IterativeProcess` 之间进行对接的一个选项是用 Python 构造 `tf.data.Datasets`，然后将这些数据集传递给 TFF。

请注意，如果我们使用经过预处理的 `ClientData`，则我们生成的数据集是上面定义的模型所期望的适当类型。

In [7]:
selected_client_ids = preprocessed_and_shuffled.client_ids[:10]

preprocessed_data_for_clients = [
    preprocessed_and_shuffled.create_tf_dataset_for_client(
        selected_client_ids[i]) for i in range(10)
]

state = trainer.initialize()
for _ in range(5):
  t1 = time.time()
  result = trainer.next(state, preprocessed_data_for_clients)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  t2 = time.time()
  print('loss {}, round time {}'.format(train_metrics['loss'], t2 - t1))

loss 2.934802532196045, round time 2.5420753955841064
loss 3.350963830947876, round time 0.45527172088623047
loss 3.1382687091827393, round time 0.47087883949279785
loss 3.0774152278900146, round time 0.4089682102203369
loss 2.9193594455718994, round time 0.3964221477508545


然而，如果我们采取这条路线，我们将***无法轻松地转移到多机模拟***。我们在本地 TensorFlow 运行时中构建的数据集可以*从周围的 Python 环境中捕获状态*，并且当它们尝试引用不再对它们可用的状态时，序列化或反序列化会失败。例如，这可能表现在 TensorFlow 的 `tensor_util.cc` 的难以理解的错误中：

```
Check failed: DT_VARIANT == input.dtype() (21 vs. 20)
```

### 客户端上的映射构造和预处理

为避免此问题，TFF 建议用户将数据集实例化和预处理视为*在每个客户端本地发生的事情*，并使用 TFF 的辅助函数或`federated_map` 在每个客户端上显式运行此预处理代码。

从概念上讲，选择这样做的原因很明显：在 TFF 的本地运行时中，由于整个联合编排发生在一台计算机上，客户端只会“偶然”访问全局 Python 环境。在这一点上值得注意的是，类似的思想导致了 TFF 的跨平台、始终可序列化的、函数式哲学。

TFF 通过 `ClientData` 的特性 `dataset_computation` 来简化这种更改，`tff.Computation` 接受 `client_id` 并返回关联的 `tf.data.Dataset`。

请注意，`preprocess` 仅处理 `dataset_computation`；预处理后的 `ClientData` 的 `dataset_computation` 特性会合并我们刚刚定义的整个预处理流水线：

In [8]:
print('dataset computation without preprocessing:')
print(client_data.dataset_computation.type_signature)
print('\n')
print('dataset computation with preprocessing:')
print(preprocessed_and_shuffled.dataset_computation.type_signature)

dataset computation without preprocessing:
(string -> <label=int32,pixels=float32[28,28]>*)


dataset computation with preprocessing:
(string -> <x=float32[?,784],y=int64[?,1]>*)


我们可以调用 `dataset_computation` 并在 Python 运行时中接收一个 eager 数据集，但是当我们组合一个迭代进程或另一个计算来彻底避免在全局 eager 运行时中具体化这些数据集时，才能发挥出这种方式的真正威力。TFF 提供了一个辅助函数 [`tff.simulation.compose_dataset_computation_with_iterative_process`](https://www.tensorflow.org/federated/api_docs/python/tff/simulation/compose_dataset_computation_with_iterative_process)，该函数可用于执行此操作。

In [9]:
trainer_accepting_ids = tff.simulation.compose_dataset_computation_with_iterative_process(
    preprocessed_and_shuffled.dataset_computation, trainer)

这个 `tff.templates.IterativeProcesses` 和上面的进程的运行方式相同；但前者接受经过预处理的客户端数据集，而后者接受表示客户端 ID 的字符串，在其主体中处理数据集构造和预处理。实际上，`state` 可以在二者之间传递。

In [10]:
for _ in range(5):
  t1 = time.time()
  result = trainer_accepting_ids.next(state, selected_client_ids)
  state = result.state
  train_metrics = result.metrics['client_work']['train']
  t2 = time.time()
  print('loss {}, round time {}'.format(train_metrics['loss'], t2 - t1))

loss 2.6114611625671387, round time 1.4935951232910156
loss 2.612247943878174, round time 0.30751872062683105
loss 2.8368589878082275, round time 0.3043978214263916
loss 2.6863903999328613, round time 0.3107311725616455
loss 2.6816341876983643, round time 0.4325370788574219


### 扩展到大量客户端

可以在 TFF 的多机运行时中立即使用 `trainer_accepting_ids`，并且避免具体化 `tf.data.Datasets` 和控制器（因此对其进行序列化并将它们发送给工作进程）。

这显著加快了分布式模拟的速度，特别是在存在大量客户端的情况下，并使中间聚合能够避免类似的序列化/反序列化开销。


### 可选的深入了解：在 TFF 中手动组合预处理逻辑

TFF 从一开始就是为组合而设计；TFF 的辅助函数刚刚执行的组合完全在用户的控制之下。我们可以使用 trainer 自己的 `next` 非常简单地手动组合我们刚刚定义的预处理计算：

In [11]:
selected_clients_type = tff.FederatedType(preprocessed_and_shuffled.dataset_computation.type_signature.parameter, tff.CLIENTS)

@tff.federated_computation(trainer.next.type_signature.parameter[0], selected_clients_type)
def new_next(server_state, selected_clients):
  preprocessed_data = tff.federated_map(preprocessed_and_shuffled.dataset_computation, selected_clients)
  return trainer.next(server_state, preprocessed_data)

manual_trainer_with_preprocessing = tff.templates.IterativeProcess(initialize_fn=trainer.initialize, next_fn=new_next)

实际上，这就是我们使用的辅助函数在底层所做的工作（外加执行适当的类型检查和操作）。我们甚至可以用略微不同的方式表达相同的逻辑，方法是将 `preprocess_and_shuffle` 序列化为一个 `tff.Computation`，并将 `federated_map` 分解为两个步骤，一个步骤会构造未经预处理的数据集，另一个步骤会在每个客户端运行 `preprocess_and_shuffle`。

我们可以验证这个更加手动的路径能够得到与 TFF 的辅助函数（模参数名称）具有相同类型签名的计算：

In [12]:
print(trainer_accepting_ids.next.type_signature)
print(manual_trainer_with_preprocessing.next.type_signature)

(<state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,client_data={string}@CLIENTS> -> <state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,metrics=<distributor=<>,client_work=<train=<sparse_categorical_accuracy=float32,loss=float32,num_examples=int64,num_batches=int64>>,aggregator=<mean_value=<>,mean_weight=<>>,finalizer=<>>@SERVER>)
(<server_state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggregator=<value_sum_process=<>,weight_sum_process=<>>,finalizer=<int64>>@SERVER,selected_clients={string}@CLIENTS> -> <state=<global_model_weights=<trainable=<float32[784,10],float32[10]>,non_trainable=<>>,distributor=<>,client_work=<>,aggre